In [4]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from numpy import sqrt
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests


In [5]:
net= pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\A Proteomic Network\A prot net start.xlsx", 
                    header=0)
net

,Gene names,Control 1,Control 2,Control 3,Control 4,Control 5,Control 6,Control 7,Control 8,ALS 1,...,ALS 10,ALS 11,ALS 12,ALS 13,ALS 14,ALS 15,ALS 16,ALS 17,ALS 18,ALS 19
0,TUBA1A,35.340465,35.360227,35.316990,35.629164,35.306858,35.208816,35.252199,35.352975,35.328403,...,35.296500,35.200008,35.360328,35.456076,35.231312,35.318894,35.317382,35.210471,35.329186,35.180793
1,HBB,34.805838,34.799389,34.960708,35.679915,35.640050,34.863775,34.106963,34.183703,35.292579,...,35.074720,35.844375,35.704953,35.419922,35.570901,34.808487,35.454242,35.192410,34.957870,35.278811
2,TUBB4B,35.237408,35.229361,35.225127,35.423859,35.175022,35.164483,35.218322,35.250528,35.240021,...,35.251757,35.170261,35.224068,35.276747,35.099084,35.209713,35.198222,35.087939,35.223477,35.050758
3,ACTG1,35.080176,35.035111,35.046158,35.221391,34.970036,35.054252,35.040957,35.133440,35.075034,...,35.018077,35.019814,34.988880,34.981886,34.995591,35.036826,35.145769,35.015405,35.052608,34.940533
4,HBA2,34.441864,34.415415,34.430065,35.063952,35.273882,34.427993,33.929326,33.854507,34.813764,...,34.693177,35.302087,35.275072,34.907987,35.146757,34.452034,35.023081,34.766622,34.522237,34.913918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2607,MEAF6,18.180863,18.275198,18.508010,18.271233,18.565836,18.541315,18.799742,18.990770,18.186564,...,18.239559,18.066484,18.532285,18.596379,18.356034,18.305115,18.539767,18.727299,17.951605,18.684145
2608,ACBD5,18.497584,18.438914,18.571335,19.681870,18.498736,18.582755,19.420220,18.969912,18.394302,...,18.521256,18.639366,18.962521,18.913779,19.098631,19.086600,18.649882,18.985133,18.200248,18.705306
2609,SLC4A8,19.214524,18.096978,18.349789,18.591834,18.707311,18.852882,20.596735,20.295215,18.207397,...,17.828971,21.733180,19.215550,17.748237,18.196099,19.350617,18.976159,20.869254,19.437182,20.768737
2610,PPIG,18.211645,18.170713,18.864304,19.514376,18.430147,18.518114,19.229856,19.334142,18.830838,...,18.546334,18.127563,18.846674,18.258606,19.027720,19.135276,18.452511,19.246079,17.839773,18.437862


In [6]:
# Automatically identify group columns by their prefixes
group1= net.loc[:, net.columns.str.startswith('Control ')]
group2= net.loc[:, net.columns.str.startswith('ALS ')]

In [7]:
group1.shape


(2612, 8)

In [8]:
# Perform Mann-Whitney U-test row-wise, drop NA values 
#stat, p_value = mannwhitneyu(group1.iloc[0], group2.iloc[0], alternative='two-sided', method='auto')

stats = []
p_vals = []
for i in range(group1.shape[0]):
    stat, p_value = mannwhitneyu(group1.iloc[i].dropna(), group2.iloc[i].dropna(), alternative='two-sided', method='auto')
    stats.append(stat)
    p_vals.append(p_value)

#print(f"U-statistic: {stat}, p-value: {p_value}")

In [9]:
net['U-stat'] = stats
net['P-value'] = p_vals
net.head()

,Gene names,Control 1,Control 2,Control 3,Control 4,Control 5,Control 6,Control 7,Control 8,ALS 1,...,ALS 12,ALS 13,ALS 14,ALS 15,ALS 16,ALS 17,ALS 18,ALS 19,U-stat,P-value
0,TUBA1A,35.340465,35.360227,35.316990,35.629164,35.306858,35.208816,35.252199,35.352975,35.328403,...,35.360328,35.456076,35.231312,35.318894,35.317382,35.210471,35.329186,35.180793,77.0,0.979267
1,HBB,34.805838,34.799389,34.960708,35.679915,35.640050,34.863775,34.106963,34.183703,35.292579,...,35.704953,35.419922,35.570901,34.808487,35.454242,35.192410,34.957870,35.278811,35.0,0.029322
2,TUBB4B,35.237408,35.229361,35.225127,35.423859,35.175022,35.164483,35.218322,35.250528,35.240021,...,35.224068,35.276747,35.099084,35.209713,35.198222,35.087939,35.223477,35.050758,90.0,0.481460
3,ACTG1,35.080176,35.035111,35.046158,35.221391,34.970036,35.054252,35.040957,35.133440,35.075034,...,34.988880,34.981886,34.995591,35.036826,35.145769,35.015405,35.052608,34.940533,92.0,0.418424
4,HBA2,34.441864,34.415415,34.430065,35.063952,35.273882,34.427993,33.929326,33.854507,34.813764,...,35.275072,34.907987,35.146757,34.452034,35.023081,34.766622,34.522237,34.913918,29.0,0.011246


In [10]:
# Apply Benjamini-Hochberg correction (optional)
_, p_adjusted, _, _ = multipletests(net['P-value'], method='fdr_bh')
net['p_adjusted'] = p_adjusted
net.head()

,Gene names,Control 1,Control 2,Control 3,Control 4,Control 5,Control 6,Control 7,Control 8,ALS 1,...,ALS 13,ALS 14,ALS 15,ALS 16,ALS 17,ALS 18,ALS 19,U-stat,P-value,p_adjusted
0,TUBA1A,35.340465,35.360227,35.316990,35.629164,35.306858,35.208816,35.252199,35.352975,35.328403,...,35.456076,35.231312,35.318894,35.317382,35.210471,35.329186,35.180793,77.0,0.979267,1.0
1,HBB,34.805838,34.799389,34.960708,35.679915,35.640050,34.863775,34.106963,34.183703,35.292579,...,35.419922,35.570901,34.808487,35.454242,35.192410,34.957870,35.278811,35.0,0.029322,1.0
2,TUBB4B,35.237408,35.229361,35.225127,35.423859,35.175022,35.164483,35.218322,35.250528,35.240021,...,35.276747,35.099084,35.209713,35.198222,35.087939,35.223477,35.050758,90.0,0.481460,1.0
3,ACTG1,35.080176,35.035111,35.046158,35.221391,34.970036,35.054252,35.040957,35.133440,35.075034,...,34.981886,34.995591,35.036826,35.145769,35.015405,35.052608,34.940533,92.0,0.418424,1.0
4,HBA2,34.441864,34.415415,34.430065,35.063952,35.273882,34.427993,33.929326,33.854507,34.813764,...,34.907987,35.146757,34.452034,35.023081,34.766622,34.522237,34.913918,29.0,0.011246,1.0


In [11]:
# Apply Bonferroni correction (optional)
_, p_adjusted, _, _ = multipletests(net['P-value'], method='bonferroni')
net['p_adjusted_bonf'] = p_adjusted
net.head()

,Gene names,Control 1,Control 2,Control 3,Control 4,Control 5,Control 6,Control 7,Control 8,ALS 1,...,ALS 14,ALS 15,ALS 16,ALS 17,ALS 18,ALS 19,U-stat,P-value,p_adjusted,p_adjusted_bonf
0,TUBA1A,35.340465,35.360227,35.316990,35.629164,35.306858,35.208816,35.252199,35.352975,35.328403,...,35.231312,35.318894,35.317382,35.210471,35.329186,35.180793,77.0,0.979267,1.0,1.0
1,HBB,34.805838,34.799389,34.960708,35.679915,35.640050,34.863775,34.106963,34.183703,35.292579,...,35.570901,34.808487,35.454242,35.192410,34.957870,35.278811,35.0,0.029322,1.0,1.0
2,TUBB4B,35.237408,35.229361,35.225127,35.423859,35.175022,35.164483,35.218322,35.250528,35.240021,...,35.099084,35.209713,35.198222,35.087939,35.223477,35.050758,90.0,0.481460,1.0,1.0
3,ACTG1,35.080176,35.035111,35.046158,35.221391,34.970036,35.054252,35.040957,35.133440,35.075034,...,34.995591,35.036826,35.145769,35.015405,35.052608,34.940533,92.0,0.418424,1.0,1.0
4,HBA2,34.441864,34.415415,34.430065,35.063952,35.273882,34.427993,33.929326,33.854507,34.813764,...,35.146757,34.452034,35.023081,34.766622,34.522237,34.913918,29.0,0.011246,1.0,1.0


In [12]:
#df['average'] = df.mean(numeric_only=True, axis=1)
net['Mean Control'] = net.loc[:, net.columns.str.startswith('Control ')].mean(axis=1)
net['Mean ALS'] = net.loc[:, net.columns.str.startswith('ALS ')].mean(axis=1)

In [13]:
#Fold Change
net["FC ALS-C"]= net["Mean ALS"]/net["Mean Control"]

In [14]:
#Log 2 FC
net["Log2FC ALS-C"]=np.log2(net["FC ALS-C"])

In [15]:
net.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\A Proteomic Network\A prot net Mann-W.xlsx")

In [12]:
# Sort the DataFrame by the absolute values of the column
net_sorted = net.loc[net["Log2FC ALS-C"].abs().sort_values(ascending=False).index]



In [13]:
net_sorted.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\A Proteomic Network\A prot net Mann-W sorted.xlsx")

In [14]:
# Filter rows where 'P-value' <= 0.05
filtered_net = net_sorted[net_sorted['P-value'] <= 0.05]

In [16]:
filtered_net.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\A Proteomic Network\A prot net Mann-W filtered.xlsx")